<a href="https://colab.research.google.com/github/NicknameSoHard/Digit-Recognizer-with-Google-Inception/blob/master/Digit_Recognizer_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Страница сорревнования Kaggle Digit Recognizer](https://www.kaggle.com/c/digit-recognizer/submissions?sortBy=date&group=all&page=1)
![alt text](https://pp.userapi.com/c849328/v849328867/dc537/3QQ0lXDlJX0.jpg)

Основной моей идеей в этом сорревновании было изучить строение нейросетей Inceprion от Google  и попробовать применить её миниатюрную версию для данного сорревнования. Ну что же, попытка не пыткаю 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp ./gdrive/'My Drive'/Kaggle/kaggle.json ~/.kaggle/
!kaggle competitions download -c digit-recognizer

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 93% 68.0M/73.2M [00:00<00:00, 44.9MB/s]
100% 73.2M/73.2M [00:00<00:00, 79.4MB/s]
 76% 37.0M/48.8M [00:00<00:00, 25.0MB/s]
100% 48.8M/48.8M [00:00<00:00, 66.8MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 75.9MB/s]


In [0]:
from pandas import read_csv

data_train = read_csv('train.csv')
data_test = read_csv('test.csv')

In [0]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
Y_train = data_train["label"]
Y_train = to_categorical(Y_train, num_classes = 10)

X_train = data_train.drop(labels = ["label"],axis = 1) 
X_train = X_train / 255.0
X_train = X_train.values.reshape(-1,28,28)

X_test = data_test / 255.0
X_test = X_test.values.reshape(-1,28,28)

Using TensorFlow backend.


In [0]:
print('Размер выборки Train:', X_train.shape)
print('Размер выбоки Test:', X_test.shape)

Размер выборки Train: (42000, 28, 28)
Размер выбоки Test: (28000, 28, 28)


In [0]:
# Начинаем строить модель
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.layers import concatenate
from keras.models import Model
from keras.optimizers import SGD

# Создаем входной слой
input_img = Input(shape = X_train[0].shape )

# Создаем блок инсепшон в2, используя свертки 1х1, 3х3, 5х5 по 64 каждого и усредняющий слой
inceprion_0 = Conv1D(64, 1, padding='same', activation='relu')(input_img)
inceprion_0 = Conv1D(64, 3, padding='same', activation='relu')(inceprion_0)
inceprion_0 = Conv1D(64, 3, padding='same', activation='relu')(inceprion_0)

inceprion_1 = Conv1D(64, 1, padding='same', activation='relu')(input_img)
inceprion_1 = Conv1D(64, 3, padding='same', activation='relu')(inceprion_1)

inceprion_2 = MaxPooling1D(3, strides=1, padding='same')(input_img)

output = concatenate([inceprion_0, inceprion_1, inceprion_2], axis = 2)
output = Flatten()(output)
out = Dense( 10, activation='softmax')(output)
model = Model(inputs = input_img, outputs = out)

# собираем модель чтоб потерять веса и переобучаем модельку
sgd = SGD(lr=0.01, momentum=0, decay=0, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
# Начинаем строить модель
from keras.layers import Input, Conv1D, MaxPooling1D
from keras.layers import concatenate

# Создаем сходной слой
input_img = Input(shape = X_train[0].shape )

# Создадим три последовательных слоя свертки 3х3, 3х3 и пулинг слой свертки
layer_1 = Conv1D(32, 3, padding='same', activation='relu')(input_img)
layer_1 = Conv1D(64, 3, padding='same', activation='relu')(layer_1)
layer_1 = MaxPooling1D(2, strides=1, padding='same')(layer_1)

# Создаем блок инсепшон, используя свертки 1х1, 3х3, 5х5 по 64 каждого и усредняющий слой
inceprion_0 = Conv1D(64, 1, padding='same', activation='relu')(layer_1)
inceprion_0 = Conv1D(64, 3, padding='same', activation='relu')(inceprion_0)
inceprion_0 = Conv1D(64, 3, padding='same', activation='relu')(inceprion_0)

inceprion_1 = Conv1D(64, 1, padding='same', activation='relu')(layer_1)
inceprion_1 = Conv1D(64, 3, padding='same', activation='relu')(inceprion_1)

inceprion_2 = MaxPooling1D(2, strides=1, padding='same')(layer_1)

inceprion = concatenate([inceprion_0, inceprion_1, inceprion_2], axis = 1)

# Создадим выходной слой нашей сети и собираем модельку
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model

inceprion = Flatten()(inceprion)

out = Dense(10, activation='softmax')(inceprion)

model = Model(inputs = input_img, outputs = out)

# Добавляем SGD оптимизатор
from keras.optimizers import SGD

sgd = SGD(lr=0.01, momentum=0, decay=0, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
model.fit(X_train, Y_train, epochs=35, batch_size=32)

Epoch 1/35
42000/42000 [==============================] - 14s 329us/step - loss: 0.5924 - acc: 0.8353
Epoch 2/35
42000/42000 [==============================] - 14s 322us/step - loss: 0.2270 - acc: 0.9332
Epoch 3/35
42000/42000 [==============================] - 13s 319us/step - loss: 0.1650 - acc: 0.9516
Epoch 4/35
42000/42000 [==============================] - 13s 318us/step - loss: 0.1323 - acc: 0.9613
Epoch 5/35
42000/42000 [==============================] - 13s 319us/step - loss: 0.1122 - acc: 0.9669
Epoch 6/35
42000/42000 [==============================] - 13s 316us/step - loss: 0.0973 - acc: 0.9715
Epoch 7/35
42000/42000 [==============================] - 13s 316us/step - loss: 0.0884 - acc: 0.9737
Epoch 8/35
42000/42000 [==============================] - 13s 318us/step - loss: 0.0798 - acc: 0.9761
Epoch 9/35
42000/42000 [==============================] - 13s 317us/step - loss: 0.0727 - acc: 0.9789
Epoch 10/35
42000/42000 [==============================] - 13s 316us/step - loss: 

In [0]:
scores = model.evaluate(X_train, Y_train, verbose=0)
print("Accuracy on MNIST:", scores[1]*100)

Accuracy on MNIST: 99.55952380952381


In [0]:
# Сохраняем результат
final_result = model.predict(X_test)

In [0]:
import numpy as np
import pandas as pd

# Подаем в нашу модель тестовые значения
final_result = model.predict(X_test)

# Сохраняем
results = np.argmax(final_result,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_result.csv",index=False)

In [0]:
kaggle competitions submit -c digit-recognizer -f cnn_mnist_result.csv -m "final solution"

Текущая модель позволила достить на тестовых данных точности **0.98371**

In [0]:
from google.colab import files

files.download('cnn_mnist_datagen.csv')